# Modelo Fraiman

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set a seed for reproducibility
np.random.seed(42)

# Parameters from the paper's simulation (Fig. 1A)
N = 1000
mu = 0
sigma = 0.3

# Generate a sequence of 1000 bids from a Log-Normal distribution
bids = np.random.uniform(0, 1, N)

# Initialize lists to store accepted and non-accepted bids
accepted_bids = []
non_accepted_bids = []
accepted_bid_indices = []
non_accepted_bid_indices = []
pending_bids = []

# Implement the auction mechanism from the paper
for i, new_bid in enumerate(bids):
    bid_index = i + 1

    # If there are no pending bids, the new bid is just added to the pending queue
    if not pending_bids:
        pending_bids.append(new_bid)
        continue

    # Find the maximum value among the pending bids
    max_pending_bid = max(pending_bids)

    # Apply the selling rule:
    if new_bid < max_pending_bid:
        # A sale occurs. Find the index of the accepted bid
        accepted_bid_index = np.where(bids == max_pending_bid)[0][0] + 1
        accepted_bids.append(max_pending_bid)
        accepted_bid_indices.append(accepted_bid_index)
        
        # The accepted bid is removed from the pending list
        pending_bids.remove(max_pending_bid)
        
        # The current bid is not accepted at this time and becomes a new pending bid
        non_accepted_bids.append(new_bid)
        non_accepted_bid_indices.append(bid_index)
        pending_bids.append(new_bid)
    else:
        # No sale occurs, the new bid is added to the pending list
        pending_bids.append(new_bid)
        non_accepted_bids.append(new_bid)
        non_accepted_bid_indices.append(bid_index)

# After the loop, any remaining bids are "frozen" and not accepted
for bid in pending_bids:
    non_accepted_bids.append(bid)
    non_accepted_bid_indices.append(np.where(bids == bid)[0][0] + 1)
    
# Get the indices and values for plotting
accepted_indices_plot = [i for i, b in enumerate(bids) if b in accepted_bids]
non_accepted_indices_plot = [i for i, b in enumerate(bids) if b not in accepted_bids]

accepted_values_plot = [bids[i] for i in accepted_indices_plot]
non_accepted_values_plot = [bids[i] for i in non_accepted_indices_plot]

accepted_indices_plot = [i + 1 for i in accepted_indices_plot]
non_accepted_indices_plot = [i + 1 for i in non_accepted_indices_plot]

# Create the scatter plot
plt.figure(figsize=(10, 6))

# Configurar el estilo del gráfico
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.scatter(non_accepted_indices_plot, non_accepted_values_plot, c='black', label='Not Accepted', s=20, alpha=0.3)
plt.scatter(accepted_indices_plot, accepted_values_plot, c='red', label='Accepted', s=20, alpha=0.3)

# Add critical price line based on the paper's value
critical_price = 1/np.e
plt.axhline(y=critical_price, color='#800080', linestyle=':', linewidth=3.5, 
            dashes=(8, 3), alpha=0.9)
# Agregar texto al lado de la línea
plt.text(1060, critical_price, f'1/e ≈ {round(critical_price, 4)}', 
         color='#800080', fontsize=11, verticalalignment='center')

plt.suptitle('Simulación del proceso de subasta', 
           fontsize=16, fontweight='bold', y=0.98)
plt.title('1000 postores siguiendo una distribución uniforme entre 0 y 1.\n' +
         'Se observa la generación espontánea de un precio crítico entorno a 1/e.',
         pad=20, fontsize=11,y=0.94)
plt.xlabel('Bidder')
plt.ylabel('Bid [$]')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=11)

plt.show()
#plt.savefig('auction_scatter_plot.png')
#print("Plot saved to auction_scatter_plot.png")

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Set a seed for reproducibility
np.random.seed(42)

# Parameters
N = 1000

# Generate a sequence of 1000 bids from a uniform distribution
bids = np.random.uniform(0, 1, N)

# Initialize lists to store accepted and non-accepted bids
accepted_bids = []
accepted_bid_indices = []
pending_bids = []

# For animation: keep track of which bids are accepted at each step
frames_data = []

# Implement the auction mechanism and record animation frames
for i, new_bid in enumerate(bids):
    bid_index = i + 1

    # If there are no pending bids, the new bid is just added to the pending queue
    if not pending_bids:
        pending_bids.append((new_bid, bid_index))
        # For animation: no accepted bids yet
        accepted_mask = [False] * (i + 1)
        frames_data.append(accepted_mask)
        continue

    # Find the maximum value among the pending bids
    max_pending_bid, max_pending_idx = max(pending_bids, key=lambda x: x[0])

    # Apply the selling rule:
    if new_bid < max_pending_bid:
        # A sale occurs. Find the index of the accepted bid
        accepted_bids.append(max_pending_bid)
        accepted_bid_indices.append(max_pending_idx)
        # The accepted bid is removed from the pending list
        pending_bids = [b for b in pending_bids if b[0] != max_pending_bid or b[1] != max_pending_idx]
        # The current bid is not accepted at this time and becomes a new pending bid
        pending_bids.append((new_bid, bid_index))
    else:
        # No sale occurs, the new bid is added to the pending list
        pending_bids.append((new_bid, bid_index))

    # For animation: mark which bids have been accepted so far
    accepted_mask = [False] * (i + 1)
    for idx in accepted_bid_indices:
        if idx <= i + 1:
            accepted_mask[idx - 1] = True
    frames_data.append(accepted_mask)

# After the loop, any remaining bids are "frozen" and not accepted

# Build Plotly animation frames
scatter_marker_size = 10
frames = []
for step, accepted_mask in enumerate(frames_data):
    x = np.arange(1, step + 2)
    y = bids[:step + 1]
    colors = np.where(accepted_mask, 'red', 'black')
    frame_data = [
        go.Scatter(
            x=x,
            y=y,
            mode='markers',
            marker=dict(color=colors, size=scatter_marker_size, opacity=0.7),
            showlegend=True,
            hoverinfo='x+y',
        )
    ]
    # Only in the last frame, add the 1/e line and its label
    if step == len(frames_data) - 1:
        critical_price = 1/np.e
        frame_data.append(
            go.Scatter(
                x=[1, N],
                y=[critical_price, critical_price],
                mode='lines',
                line=dict(color='#800080', width=3, dash='dot'),
                name='1/e',
                showlegend=True
            )
        )
        frame_data.append(
            go.Scatter(
                x=[N + 20],
                y=[critical_price],
                mode='text',
                text=[f'1/e ≈ {round(critical_price, 4)}'],
                textposition='middle left',
                textfont=dict(color='#800080', size=14),
                showlegend=True,
                hoverinfo='skip'
            )
        )
    frames.append(go.Frame(data=frame_data, name=str(step)))

# Initial data for the first frame
init_x = np.arange(1, 2)
init_y = bids[:1]
init_colors = ['black']
init_data = [
    go.Scatter(
        x=init_x,
        y=init_y,
        mode='markers',
        marker=dict(color=init_colors, size=scatter_marker_size, opacity=0.7),
        showlegend=False,
        hoverinfo='x+y',
    )
]

# Layout
layout = go.Layout(
    title=dict(
        text='Simulación del proceso de subasta',
        font=dict(size=22, family='Arial', color='black'),
        x=0.5,
        y=0.93
    ),
    xaxis=dict(
        title='Bidder',
        range=[0, N + 50],
        showgrid=True,  # Cambiado a True para mostrar la cuadrícula
        zeroline=True,
        showline=True,  # Asegura que el eje X se muestre
        linecolor='black',
        linewidth=2,
        mirror=True,
        ticks='outside',
        tickfont=dict(size=14, color='black'),
        # Removed 'titlefont' as it is not a valid property
        showticklabels=True
    ),
    yaxis=dict(
        title='Bid [$]',
        range=[-0.05, 1.05],
        showgrid=True,  # Cambiado a True para mostrar la cuadrícula
        zeroline=True,
        showline=True,  # Asegura que el eje Y se muestre
        linecolor='black',
        linewidth=2,
        mirror=True,
        ticks='outside',
        tickfont=dict(size=14, color='black'),
        # Removed 'titlefont' as it is not a valid property
        showticklabels=True
    ),
    plot_bgcolor='white',  # Saca el fondo gris
    paper_bgcolor='white', # Saca el fondo gris
    updatemenus=[
        dict(
            type='buttons',
            showactive=False,
            y=0.05,   # Más abajo
            x=1.35,   # Más a la derecha
            xanchor='right',
            yanchor='bottom',
            buttons=[
                dict(label='Play',
                     method='animate',
                     args=[None, dict(frame=dict(duration=10, redraw=True), fromcurrent=True, mode='immediate')]),
                dict(label='Pause',
                     method='animate',
                     args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])
            ]
        )
    ],
    # Remove subtitle/annotation
    annotations=[],
    legend=dict(
        x=1.02, y=1, xanchor='left', yanchor='top', font=dict(size=13)
    ),
    margin=dict(l=60, r=180, t=80, b=60)
)

fig = go.Figure(
    data=init_data,
    layout=layout,
    frames=frames
)

fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Set a seed for reproducibility
np.random.seed(42)

# Parameters
N = 1000

# Generate a sequence of 1000 bids from a uniform distribution
bids = np.random.uniform(0, 1, N)

# Initialize lists to store accepted and non-accepted bids
accepted_bids = []
accepted_bid_indices = []
pending_bids = []

# For animation: keep track of which bids are accepted at each step
frames_data = []

# Implement the auction mechanism and record animation frames
for i, new_bid in enumerate(bids):
    bid_index = i + 1

    # If there are no pending bids, the new bid is just added to the pending queue
    if not pending_bids:
        pending_bids.append((new_bid, bid_index))
        # For animation: no accepted bids yet
        accepted_mask = [False] * (i + 1)
        frames_data.append(accepted_mask)
        continue

    # Find the maximum value among the pending bids
    max_pending_bid, max_pending_idx = max(pending_bids, key=lambda x: x[0])

    # Apply the selling rule:
    if new_bid < max_pending_bid:
        # A sale occurs. Find the index of the accepted bid
        accepted_bids.append(max_pending_bid)
        accepted_bid_indices.append(max_pending_idx)
        # The accepted bid is removed from the pending list
        pending_bids = [b for b in pending_bids if b[0] != max_pending_bid or b[1] != max_pending_idx]
        # The current bid is not accepted at this time and becomes a new pending bid
        pending_bids.append((new_bid, bid_index))
    else:
        # No sale occurs, the new bid is added to the pending list
        pending_bids.append((new_bid, bid_index))

    # For animation: mark which bids have been accepted so far
    accepted_mask = [False] * (i + 1)
    for idx in accepted_bid_indices:
        if idx <= i + 1:
            accepted_mask[idx - 1] = True
    frames_data.append(accepted_mask)

# After the loop, any remaining bids are "frozen" and not accepted

# Build Plotly animation frames
scatter_marker_size = 10
frames = []
for step, accepted_mask in enumerate(frames_data):
    x = np.arange(1, step + 2)
    y = bids[:step + 1]
    # Split into two traces for legend: accepted (red) and not accepted (black)
    accepted_x = x[accepted_mask]
    accepted_y = y[accepted_mask]
    not_accepted_x = x[~np.array(accepted_mask)]
    not_accepted_y = y[~np.array(accepted_mask)]
    frame_data = []
    # Only show legend in the last frame
    show_legend = (step == len(frames_data) - 1)
    # Bids concretadas (rojo)
    frame_data.append(
        go.Scatter(
            x=accepted_x,
            y=accepted_y,
            mode='markers',
            marker=dict(color='red', size=scatter_marker_size, opacity=0.7),
            name='Bids concretadas (rojo)',
            showlegend=True,
            hoverinfo='x+y',
        )
    )
    # Bids no concretadas (negro)
    frame_data.append(
        go.Scatter(
            x=not_accepted_x,
            y=not_accepted_y,
            mode='markers',
            marker=dict(color='black', size=scatter_marker_size, opacity=0.7),
            name='Bids no concretadas (negro)',
            showlegend=True,
            hoverinfo='x+y',
        )
    )
    # Only in the last frame, add the 1/e line and its label
    if show_legend:
        critical_price = 1/np.e
        frame_data.append(
            go.Scatter(
                x=[1, N],
                y=[critical_price, critical_price],
                mode='lines',
                line=dict(color='#800080', width=3, dash='dot'),
                name='1/e',
                showlegend=True
            )
        )
        frame_data.append(
            go.Scatter(
                x=[N + 20],
                y=[critical_price],
                mode='text',
                text=[f'1/e ≈ {round(critical_price, 4)}'],
                textposition='middle left',
                textfont=dict(color='#800080', size=14),
                showlegend=True,
                hoverinfo='skip'
            )
        )
    frames.append(go.Frame(data=frame_data, name=str(step)))

# Initial data for the first frame (no legend)
init_x = np.arange(1, 2)
init_y = bids[:1]
init_accepted_mask = np.array([False])
init_data = [
    go.Scatter(
        x=init_x[init_accepted_mask],
        y=init_y[init_accepted_mask],
        mode='markers',
        marker=dict(color='red', size=scatter_marker_size, opacity=0.7),
        name='Bids concretadas (rojo)',
        showlegend=True,
        hoverinfo='x+y',
    ),
    go.Scatter(
        x=init_x[~init_accepted_mask],
        y=init_y[~init_accepted_mask],
        mode='markers',
        marker=dict(color='black', size=scatter_marker_size, opacity=0.7),
        name='Bids no concretadas (negro)',
        showlegend=True,
        hoverinfo='x+y',
    )
]

# Layout
layout = go.Layout(
    title=dict(
        text='Simulación del proceso de subasta',
        font=dict(size=22, family='Arial', color='black'),
        x=0.5,
        y=0.93
    ),
    xaxis=dict(
        title='Bidder',
        range=[0, N + 50],
        showgrid=True,
        zeroline=True,
        showline=True,
        linecolor='black',
        linewidth=2,
        mirror=False,  # Cambiado a False para quitar el marco
        ticks='outside',
        tickfont=dict(size=14, color='black'),
        showticklabels=True
    ),
    yaxis=dict(
        title='Bid [$]',
        range=[-0.05, 1.05],
        showgrid=True,
        zeroline=True,
        showline=True,
        linecolor='black',
        linewidth=2,
        mirror=False,  # Cambiado a False para quitar el marco
        ticks='outside',
        tickfont=dict(size=14, color='black'),
        showticklabels=True
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    # Configuración para quitar el marco del gráfico
    xaxis_showspikes=False,
    yaxis_showspikes=False,
    updatemenus=[
        dict(
            type='buttons',
            showactive=False,
            y=0.05,
            x=1.35,
            xanchor='right',
            yanchor='bottom',
            buttons=[
                dict(label='Play',
                     method='animate',
                     args=[None, dict(frame=dict(duration=10, redraw=True), fromcurrent=True, mode='immediate')]),
                dict(label='Pause',
                     method='animate',
                     args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')])
            ]
        )
    ],
    annotations=[],
    legend=dict(
        x=1.02, y=1, xanchor='left', yanchor='top', font=dict(size=13)
    ),
    margin=dict(l=60, r=180, t=80, b=60)
)

fig = go.Figure(
    data=init_data,
    layout=layout,
    frames=frames
)

fig.show()


# Modificación de m

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set a seed for reproducibility
np.random.seed(42)

# Parameters
N = 1000
m_values = [1, 2, 3, 5, 10, 50, 75]  # Diferentes valores de "paciencia"

# Generar una secuencia de 1000 ofertas uniformes entre 0 y 1 (misma para todos los m)
bids = np.random.uniform(0, 1, N)

# Crear subplots con Plotly
rows = 2
cols = 4
fig = make_subplots(
    rows=rows, cols=cols,
    subplot_titles=[f"m = {m}" for m in m_values] + [""] * (rows * cols - len(m_values)),
    horizontal_spacing=0.08, vertical_spacing=0.15
)

for idx, m in enumerate(m_values):
    # Inicializar listas para almacenar las ofertas aceptadas y no aceptadas
    accepted_bids = []
    non_accepted_bids = []
    accepted_bid_indices = []
    non_accepted_bid_indices = []

    if m == 1:
        # Para m=1, la oferta pendiente se acepta si la siguiente es menor o igual,
        # y se rechaza si la siguiente es mayor (la nueva se vuelve pendiente).
        pending_bid = None
        pending_idx = None
        for i, new_bid in enumerate(bids):
            bid_index = i + 1
            if pending_bid is None:
                pending_bid = new_bid
                pending_idx = bid_index
            else:
                if new_bid > pending_bid:
                    # La pendiente es rechazada, la nueva se vuelve pendiente
                    non_accepted_bids.append(pending_bid)
                    non_accepted_bid_indices.append(pending_idx)
                    pending_bid = new_bid
                    pending_idx = bid_index
                else:
                    # La pendiente es aceptada, la nueva se vuelve pendiente
                    accepted_bids.append(pending_bid)
                    accepted_bid_indices.append(pending_idx)
                    pending_bid = new_bid
                    pending_idx = bid_index
        # Al final, la última pendiente no es aceptada
        if pending_bid is not None:
            non_accepted_bids.append(pending_bid)
            non_accepted_bid_indices.append(pending_idx)
    else:
        # Estructura para las ofertas pendientes: cada elemento es un dict con 'bid', 'index', 'wait'
        pending_bids = []

        for i, new_bid in enumerate(bids):
            bid_index = i + 1

            # Añadir la nueva oferta a la lista de pendientes, con contador de paciencia en 0
            pending_bids.append({'bid': new_bid, 'index': bid_index, 'wait': 0})

            # Si hay más de una oferta pendiente, incrementar el contador de paciencia de la máxima
            if len(pending_bids) > 1:
                # Encontrar la oferta máxima pendiente
                max_idx = np.argmax([b['bid'] for b in pending_bids])
                pending_bids[max_idx]['wait'] += 1

                # Si la paciencia de la máxima llegó a m, se vende
                if pending_bids[max_idx]['wait'] >= m:
                    accepted_bids.append(pending_bids[max_idx]['bid'])
                    accepted_bid_indices.append(pending_bids[max_idx]['index'])
                    # Eliminar la oferta aceptada de la lista de pendientes
                    del pending_bids[max_idx]

        # Al final, las ofertas que quedan pendientes no son aceptadas
        for b in pending_bids:
            non_accepted_bids.append(b['bid'])
            non_accepted_bid_indices.append(b['index'])

        # También, las ofertas que nunca fueron aceptadas durante el proceso
        # (es decir, todas menos las aceptadas)
        all_accepted_set = set(accepted_bid_indices)
        for i, bid in enumerate(bids):
            idx_bid = i + 1
            if idx_bid not in all_accepted_set and idx_bid not in non_accepted_bid_indices:
                non_accepted_bids.append(bid)
                non_accepted_bid_indices.append(idx_bid)

    # Preparar los datos para el gráfico
    accepted_indices_plot = accepted_bid_indices
    non_accepted_indices_plot = non_accepted_bid_indices

    accepted_values_plot = [bids[i-1] for i in accepted_indices_plot]
    non_accepted_values_plot = [bids[i-1] for i in non_accepted_indices_plot]

    # Determinar la posición del subplot
    row = idx // cols + 1
    col = idx % cols + 1

    # Agregar los puntos no aceptados (negro)
    fig.add_trace(
        go.Scatter(
            x=non_accepted_indices_plot,
            y=non_accepted_values_plot,
            mode='markers',
            marker=dict(color='black', size=6, opacity=0.6),
            name='Not Accepted',
            showlegend=(idx == 0)
        ),
        row=row, col=col
    )
    # Agregar los puntos aceptados (rojo)
    fig.add_trace(
        go.Scatter(
            x=accepted_indices_plot,
            y=accepted_values_plot,
            mode='markers',
            marker=dict(color='red', size=6, opacity=0.8),
            name='Accepted',
            showlegend=(idx == 0)
        ),
        row=row, col=col
    )
    # Línea de precio crítico (opcional, descomentar si se desea)
    # critical_price = (1/np.e)
    # fig.add_trace(
    #     go.Scatter(
    #         x=[1, N],
    #         y=[critical_price, critical_price],
    #         mode='lines',
    #         line=dict(color='purple', width=2, dash='dash'),
    #         name=f'Critical Price ({round(critical_price, 4)})',
    #         showlegend=(idx == 0)
    #     ),
    #     row=row, col=col
    # )

# Ocultar los subplots vacíos si hay menos de rows*cols
for j in range(len(m_values), rows * cols):
    row = j // cols + 1
    col = j % cols + 1
    fig.update_xaxes(visible=False, row=row, col=col)
    fig.update_yaxes(visible=False, row=row, col=col)

# Layout general
fig.update_layout(
    height=800,
    width=2000,
    title_text='Auction Process Simulation for Different Patience Values (m)',
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend=dict(
        x=1.02, y=1, xanchor='left', yanchor='top', font=dict(size=13)
    ),
    margin=dict(l=60, r=180, t=80, b=60)
)

# Ejes de cada subplot
for idx in range(len(m_values)):
    row = idx // cols + 1
    col = idx % cols + 1
    fig.update_xaxes(
        title_text='Bidder',
        range=[0, N + 50],
        showgrid=True,
        zeroline=True,
        showline=True,
        linecolor='black',
        linewidth=2,
        mirror=True,
        ticks='outside',
        tickfont=dict(size=14, color='black'),
        showticklabels=True,
        row=row, col=col
    )
    fig.update_yaxes(
        title_text='Bid [$]',
        range=[-0.05, 1.05],
        showgrid=True,
        zeroline=True,
        showline=True,
        linecolor='black',
        linewidth=2,
        mirror=True,
        ticks='outside',
        tickfont=dict(size=14, color='black'),
        showticklabels=True,
        row=row, col=col
    )

fig.show()

In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd  # <-- Agregado para el DataFrame

# Set a seed for reproducibility
np.random.seed(42)

# Parameters
N = 1000
default_m_values = [1, 2, 3, 5, 10, 50, 75]  # Valores de "paciencia" por defecto

# Generar una secuencia de 1000 ofertas uniformes entre 0 y 1 (misma para todos los m)
bids = np.random.uniform(0, 1, N)

def auction_simulation_data(m, N=N, bids=bids):
    """
    Simula el proceso de subasta para un valor dado de m (paciencia).
    Devuelve los índices y valores de aceptadas y no aceptadas.
    """
    accepted_bids = []
    non_accepted_bids = []
    accepted_bid_indices = []
    non_accepted_bid_indices = []

    if m == 1:
        # Regla correcta: una oferta se acepta si es el máximo pendiente y llega una oferta menor.
        pending_bids = []
        for i, new_bid in enumerate(bids):
            bid_index = i + 1
            if not pending_bids:
                pending_bids.append((new_bid, bid_index))
            else:
                # Buscar el máximo pendiente
                max_pending_bid, max_pending_idx = max(pending_bids, key=lambda x: x[0])
                if new_bid < max_pending_bid:
                    # Se acepta el máximo pendiente
                    accepted_bids.append(max_pending_bid)
                    accepted_bid_indices.append(max_pending_idx)
                    # Eliminar el máximo pendiente de la lista
                    pending_bids = [b for b in pending_bids if b[1] != max_pending_idx]
                    # Agregar la nueva oferta como pendiente
                    pending_bids.append((new_bid, bid_index))
                else:
                    # No se acepta nada, solo se agrega la nueva oferta como pendiente
                    pending_bids.append((new_bid, bid_index))
        # Al final, las ofertas pendientes no se aceptan
        for b, idx in pending_bids:
            non_accepted_bids.append(b)
            non_accepted_bid_indices.append(idx)
    else:
        pending_bids = []
        for i, new_bid in enumerate(bids):
            bid_index = i + 1
            pending_bids.append({'bid': new_bid, 'index': bid_index, 'wait': 0})

            if len(pending_bids) > 1:
                max_idx = np.argmax([b['bid'] for b in pending_bids])
                pending_bids[max_idx]['wait'] += 1

                if pending_bids[max_idx]['wait'] >= m:
                    accepted_bids.append(pending_bids[max_idx]['bid'])
                    accepted_bid_indices.append(pending_bids[max_idx]['index'])
                    del pending_bids[max_idx]

        for b in pending_bids:
            non_accepted_bids.append(b['bid'])
            non_accepted_bid_indices.append(b['index'])

    all_accepted_set = set(accepted_bid_indices)
    for i, bid in enumerate(bids):
        idx_bid = i + 1
        if idx_bid not in all_accepted_set and idx_bid not in non_accepted_bid_indices:
            non_accepted_bids.append(bid)
            non_accepted_bid_indices.append(idx_bid)

    accepted_indices_plot = accepted_bid_indices
    non_accepted_indices_plot = non_accepted_bid_indices

    accepted_values_plot = [bids[i-1] for i in accepted_indices_plot]
    non_accepted_values_plot = [bids[i-1] for i in non_accepted_indices_plot]

    return {
        "accepted_indices": accepted_indices_plot,
        "accepted_values": accepted_values_plot,
        "non_accepted_indices": non_accepted_indices_plot,
        "non_accepted_values": non_accepted_values_plot
    }

# --- Plotly Animation with Subplots ---

import plotly.io as pio
pio.renderers.default = "notebook_connected"  # or "colab" or "jupyterlab" as needed

# Subplot grid: 2 rows x 4 cols (for 7 m values, last subplot empty)
rows = 2
cols = 4
subplot_titles = [f"m = {m}" for m in default_m_values] + [""]

# Precompute all data for each m
all_data = [auction_simulation_data(m) for m in default_m_values]

# For animation: we will animate the "reveal" of the bids (step by step)
max_steps = N  # up to N bidders

# Create subplots
fig = make_subplots(rows=rows, cols=cols, subplot_titles=subplot_titles)

# Helper to get subplot position
def get_subplot_pos(idx):
    return (idx // cols) + 1, (idx % cols) + 1

# Initial (frame 0) traces
for idx, data in enumerate(all_data):
    row, col = get_subplot_pos(idx)
    # Only show the first point at frame 0
    fig.add_trace(
        go.Scatter(
            x=data["non_accepted_indices"][:1],
            y=data["non_accepted_values"][:1],
            mode="markers",
            marker=dict(color="black", size=6, opacity=0.6),
            name="Not Accepted",
            showlegend=(idx == 0)
        ),
        row=row, col=col
    )
    fig.add_trace(
        go.Scatter(
            x=data["accepted_indices"][:1],
            y=data["accepted_values"][:1],
            mode="markers",
            marker=dict(color="red", size=6, opacity=0.8),
            name="Accepted",
            showlegend=(idx == 0)
        ),
        row=row, col=col
    )

# Animation frames
frames = []
for step in range(1, max_steps+1, 10):  # step by 10 for speed
    frame_data = []
    for idx, data in enumerate(all_data):
        # Show all points up to current step
        # For each subplot, two traces: not accepted, accepted
        non_acc_mask = [i <= step for i in data["non_accepted_indices"]]
        acc_mask = [i <= step for i in data["accepted_indices"]]
        non_acc_x = [x for x, m in zip(data["non_accepted_indices"], non_acc_mask) if m]
        non_acc_y = [y for y, m in zip(data["non_accepted_values"], non_acc_mask) if m]
        acc_x = [x for x, m in zip(data["accepted_indices"], acc_mask) if m]
        acc_y = [y for y, m in zip(data["accepted_values"], acc_mask) if m]
        frame_data.append(go.Scatter(
            x=non_acc_x,
            y=non_acc_y,
            mode="markers",
            marker=dict(color="black", size=6, opacity=0.6),
            showlegend=False
        ))
        frame_data.append(go.Scatter(
            x=acc_x,
            y=acc_y,
            mode="markers",
            marker=dict(color="red", size=6, opacity=0.8),
            showlegend=False
        ))
    frames.append(go.Frame(data=frame_data, name=str(step)))

# Update traces for animation
fig.frames = frames

# Update layout for animation controls
fig.update_layout(
    height=800,
    width=1800,
    title_text="Auction Process Simulation for Different Patience Values (m)",
    plot_bgcolor='white',
    paper_bgcolor='white',
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            y=1.15,
            x=1.05,
            xanchor="right",
            yanchor="top",
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[
                        None,
                        dict(
                            frame=dict(duration=30, redraw=True),
                            fromcurrent=True,
                            transition=dict(duration=0)
                        )
                    ]
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[
                        [None],
                        dict(frame=dict(duration=0, redraw=False), mode="immediate", transition=dict(duration=0))
                    ]
                )
            ]
        )
    ],
    legend=dict(x=1.01, y=1, bordercolor="Black", borderwidth=1)
)

# Set axes titles and ranges, and remove grid (all white)
for idx in range(len(default_m_values)):
    row, col = get_subplot_pos(idx)
    fig.update_xaxes(
        title_text="Bidder",
        row=row,
        col=col,
        range=[0, N],
        showgrid=False,
        zeroline=False,
        showline=True,
        linecolor='black',
        linewidth=2,
        mirror=True,
        ticks='outside',
        tickfont=dict(size=14, color='black'),
        showticklabels=True
    )
    fig.update_yaxes(
        title_text="Bid [$]",
        row=row,
        col=col,
        range=[0, 1],
        showgrid=False,
        zeroline=False,
        showline=True,
        linecolor='black',
        linewidth=2,
        mirror=True,
        ticks='outside',
        tickfont=dict(size=14, color='black'),
        showticklabels=True
    )

# Hide the last empty subplot
fig.update_xaxes(visible=False, row=2, col=4)
fig.update_yaxes(visible=False, row=2, col=4)

fig.show()

# --- Agregado: DataFrame resumen de Q vendida, P promedio vendido y P*Q (Ingreso Total) ---
summary_data = []
for m, data in zip(default_m_values, all_data):
    Q_vendida = len(data["accepted_values"])
    if Q_vendida > 0:
        P_promedio = np.mean(data["accepted_values"])
        ingreso_total = np.sum(data["accepted_values"])
    else:
        P_promedio = 0
        ingreso_total = 0
    summary_data.append({
        "m": m,
        "Q_vendida": Q_vendida,
        "P_promedio_vendido": P_promedio,
        "Ingreso_total": ingreso_total
    })

df_summary = pd.DataFrame(summary_data)
display(df_summary)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def simulate_x_k_max(num_bids):
    """
    Simula el valor de X_k^max a lo largo de una secuencia de ofertas.

    Args:
        num_bids (int): Número total de ofertas a simular.

    Returns:
        list: Lista con el valor de X_k^max en cada paso k.
    """
    bids = np.random.uniform(0, 1, num_bids)
    pending_bids = []
    x_k_max_values = []
    for k in range(num_bids):
        new_bid = bids[k]
        if not pending_bids:
            pending_bids.append(new_bid)
            x_k_max_values.append(new_bid)
            continue
        current_x_k_max = max(pending_bids)
        if new_bid < current_x_k_max:
            pending_bids.remove(current_x_k_max)
        pending_bids.append(new_bid)
        new_x_k_max = max(pending_bids)
        x_k_max_values.append(new_x_k_max)
    return x_k_max_values

# Realizar muchas simulaciones y guardar el último X_k^max de cada una
num_bids = 3000
num_iteraciones = 500
ultimos_x_k_max = []

for _ in range(num_iteraciones):
    valores = simulate_x_k_max(num_bids)
    ultimos_x_k_max.append(valores[-1])

min_value = np.min(ultimos_x_k_max)
# Calcular la esperanza (media) de la distribución de X_k^max final
# Graficar el histograma de los valores finales de X_k^max
plt.figure(figsize=(10,6))
plt.hist(ultimos_x_k_max, bins=30, color='royalblue', edgecolor='black', alpha=0.7)
plt.title(f'Histograma de los valores finales de $X_k^{{max}}$ tras {num_iteraciones} iteraciones \n Minimio  ≈ {min_value:.4f}')
plt.xlabel('$X_k^{max}$ final')
plt.ylabel('Frecuencia')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

def run_auction_simulation_final(N, m):
    """
    Ejecuta una simulación de subasta con la lógica correcta del paper de Fraiman (2022).
    Un campeón se vende si llegan 'm' ofertas consecutivas de menor valor.

    Args:
        N (int): Número de postores (bidders).
        m (int): Parámetro de paciencia (número de ofertas menores consecutivas).

    Returns:
        float: El valor de la oferta máxima que queda al final del proceso (X_N^max).
    """
    # Se generan todas las ofertas al principio
    bids = np.random.uniform(0, 1, N)
    
    pending_bids = []
    patience_counter = 0
    current_max_bid = -1 # Inicializar con un valor imposible

    for new_bid in bids:
        if not pending_bids:
            # Si es la primera oferta, simplemente se añade
            pending_bids.append(new_bid)
            current_max_bid = new_bid
            continue
        
        # Lógica de paciencia
        if new_bid < current_max_bid:
            patience_counter += 1
        else: # new_bid >= current_max_bid
            patience_counter = 0
        
        # La nueva oferta siempre entra a la lista de pendientes
        pending_bids.append(new_bid)
        
        # Comprobar si se cumple la condición de venta
        if patience_counter >= m:
            # Se vende el 'current_max_bid', se elimina de la lista y se resetea el contador
            pending_bids.remove(current_max_bid)
            patience_counter = 0
        
        # Se actualiza el campeón para la siguiente iteración
        if pending_bids:
            current_max_bid = max(pending_bids)
        else:
            current_max_bid = -1

    # Devolver el máximo de las ofertas que quedaron sin vender
    if pending_bids:
        return max(pending_bids)
    else:
        return np.nan

# --- Parámetros de la Simulación ---
num_simulations = 1000 # Aumentamos para mayor precisión
N = 5000
m_values = [1, 2, 3, 4, 5, 10] # Los dos casos de interés

final_max_bids = {m: [] for m in m_values}

# --- Ejecución de las Simulaciones ---
for m in m_values:
    # Usamos tqdm para una barra de progreso
    for _ in tqdm(range(num_simulations), desc=f"Simulating for m={m}"):
        final_max = run_auction_simulation_final(N, m)
        if not np.isnan(final_max):
            final_max_bids[m].append(final_max)

# --- Visualización de Resultados ---
fig, axes = plt.subplots(1, 6, figsize=(24, 6), constrained_layout=True)

for idx, m in enumerate(m_values):
    ax = axes[idx]
    ax.hist(final_max_bids[m], bins=50, density=True, color='firebrick', alpha=0.7, label='Simulación')
    min_val = np.min(final_max_bids[m]) if len(final_max_bids[m]) > 0 else float('nan')
    ax.set_title(f'Distribución de $X_N^{{max}}$\nm = {m}\n(min = {min_val:.3f})', fontsize=13)
    ax.set_xlabel('Valor Máximo Final')
    ax.set_ylabel('Densidad')
    ax.legend()
    ax.grid(True, linestyle=':', alpha=0.6)

plt.suptitle(f'Verificación de Precios Críticos del Paper de Fraiman (2022)\n(Mostrando el mínimo en cada histograma)', fontsize=18)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

def run_auction_simulation_final(N, m):
    """
    Ejecuta una simulación de subasta con la lógica correcta del paper de Fraiman (2022).
    Un campeón se vende si llegan 'm' ofertas consecutivas de menor valor.

    Args:
        N (int): Número de postores (bidders).
        m (int): Parámetro de paciencia (número de ofertas menores consecutivas).

    Returns:
        float: El valor de la oferta máxima que queda al final del proceso (X_N^max).
    """
    # Se generan todas las ofertas al principio
    bids = np.random.uniform(0, 1, N)
    
    pending_bids = []
    patience_counter = 0
    current_max_bid = -1 # Inicializar con un valor imposible

    for new_bid in bids:
        if not pending_bids:
            # Si es la primera oferta, simplemente se añade
            pending_bids.append(new_bid)
            current_max_bid = new_bid
            continue
        
        # Lógica de paciencia
        if new_bid < current_max_bid:
            patience_counter += 1
        else: # new_bid >= current_max_bid
            patience_counter = 0
        
        # La nueva oferta siempre entra a la lista de pendientes
        pending_bids.append(new_bid)
        
        # Comprobar si se cumple la condición de venta
        if patience_counter >= m:
            # Se vende el 'current_max_bid', se elimina de la lista y se resetea el contador
            pending_bids.remove(current_max_bid)
            patience_counter = 0
        
        # Se actualiza el campeón para la siguiente iteración
        if pending_bids:
            current_max_bid = max(pending_bids)
        else:
            current_max_bid = -1

    # Devolver el máximo de las ofertas que quedaron sin vender
    if pending_bids:
        return max(pending_bids)
    else:
        return np.nan

# --- Parámetros de la Simulación ---
num_simulations = 1000 # Aumentamos para mayor precisión
N = 5000
m_values = [1, 2, 3, 4, 5, 10] # Los dos casos de interés

final_max_bids = {m: [] for m in m_values}

# --- Ejecución de las Simulaciones ---
for m in m_values:
    # Usamos tqdm para una barra de progreso
    for _ in tqdm(range(num_simulations), desc=f"Simulating for m={m}"):
        final_max = run_auction_simulation_final(N, m)
        if not np.isnan(final_max):
            final_max_bids[m].append(final_max)

# --- Visualización de Resultados ---
fig, axes = plt.subplots(1, 6, figsize=(24, 6), constrained_layout=True)

for idx, m in enumerate(m_values):
    ax = axes[idx]
    ax.hist(final_max_bids[m], bins=50, density=True, color='firebrick', alpha=0.7, label='Simulación')
    min_val = np.min(final_max_bids[m]) if len(final_max_bids[m]) > 0 else float('nan')
    ax.set_title(f'Distribución de $X_N^{{max}}$\nm = {m}\n(min = {min_val:.3f})', fontsize=13)
    ax.set_xlabel('Valor Máximo Final')
    ax.set_ylabel('Densidad')
    ax.legend()
    ax.grid(True, linestyle=':', alpha=0.6)

plt.suptitle(f'Verificación de Precios Críticos del Paper de Fraiman (2022)\n(Mostrando el mínimo en cada histograma)', fontsize=18)
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 6, figsize=(24, 6), constrained_layout=True)
for idx, m in enumerate(m_values):
    ax = axes[idx]
    ax.hist(final_max_bids[m], bins=50, density=False, color='firebrick', alpha=0.7, label='Simulación')
    min_val = np.min(final_max_bids[m]) if len(final_max_bids[m]) > 0 else float('nan')
    ax.set_title(f'Distribución de $X_N^{{max}}$\nm = {m}\n(min = {min_val:.3f})', fontsize=13)
    ax.set_xlabel('Valor Máximo Final')
    ax.set_ylabel('Densidad')
    ax.legend()
    ax.grid(True, linestyle=':', alpha=0.6)

plt.suptitle(f'Verificación de Precios Críticos del Paper de Fraiman (2022)\n(Mostrando el mínimo en cada histograma)', fontsize=18)
plt.show()


# Modifación del modelo

# Memoria finita del venedor

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def run_auction_with_finite_memory(N, memory_size):
    """
    Simula una subasta con m=1 pero con una memoria finita para encontrar el máximo.
    El máximo (campeón) se busca solo dentro de las últimas 'memory_size' ofertas activas.

    Args:
        N (int): Número total de postores.
        memory_size (int): El tamaño de la ventana de memoria para buscar el máximo.

    Returns:
        tuple: Una tupla conteniendo (lista de ofertas vendidas, lista de ofertas pendientes/no vendidas).
    """
    # Generar todas las ofertas al principio
    bids = np.random.uniform(0, 1, N)
    
    # Usaremos diccionarios para rastrear tanto el valor como el índice original de cada oferta
    pending_bids = []
    sold_bids = []

    for i, bid_value in enumerate(bids):
        new_bid = {'bid': bid_value, 'index': i}

        # Si no hay ofertas pendientes, simplemente se añade la nueva y continuamos
        if not pending_bids:
            pending_bids.append(new_bid)
            continue
            
        # --- LÓGICA DE MEMORIA FINITA ---
        # Definimos la ventana de memoria: las últimas 'memory_size' ofertas pendientes
        memory_window = pending_bids[-memory_size:]
        
        # El campeón es el máximo, pero SOLO dentro de esa ventana
        champion = max(memory_window, key=lambda x: x['bid'])
        
        # Aplicamos la regla de m=1: si la nueva oferta es menor que el campeón, se vende el campeón
        if new_bid['bid'] < champion['bid']:
            sold_bids.append(champion)
            pending_bids.remove(champion)
        
        # La nueva oferta siempre entra a la lista de pendientes
        pending_bids.append(new_bid)

    return sold_bids, pending_bids

# --- PARÁMETROS DE LA SIMULACIÓN ---
N = 1000  # Número total de ofertas
MEMORY_SIZE = 5 # Tamaño de la memoria. ¡Puedes experimentar cambiando este valor!

# --- EJECUCIÓN DE UNA ÚNICA SIMULACIÓN ---
sold, unsold = run_auction_with_finite_memory(N, MEMORY_SIZE)

# --- PREPARACIÓN DE DATOS PARA EL GRÁFICO ---
# Extraemos los índices (eje x) y los valores (eje y) de las ofertas
sold_indices = [item['index'] for item in sold]
sold_values = [item['bid'] for item in sold]

unsold_indices = [item['index'] for item in unsold]
unsold_values = [item['bid'] for item in unsold]

# --- VISUALIZACIÓN ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(16, 8))

# Graficamos las ofertas no vendidas en negro
plt.scatter(unsold_indices, unsold_values, c='black', label='No Vendidos', s=15, alpha=0.6)

# Graficamos las ofertas vendidas en rojo
plt.scatter(sold_indices, sold_values, c='red', label='Vendidos', s=25, alpha=0.8, edgecolors='black', linewidths=0.5)

plt.title(f'Simulación de Subasta con Memoria Finita (Tamaño de Memoria = {MEMORY_SIZE})', fontsize=18)
plt.xlabel('Índice del Postor (Orden de llegada)', fontsize=12)
plt.ylabel('Valor de la Oferta', fontsize=12)
plt.legend(loc='lower center', fontsize=12, bbox_to_anchor=(0.5, -0.12), ncol=2)
plt.xlim(0, N)
plt.ylim(0, 1)

plt.show()


# Calcula y muestra Q Vendida, P Promedio de venta e Ingreso total (P*Q)
import pandas as pd

Q_vendida = len(sold)
if Q_vendida > 0:
    P_promedio_vendido = sum(sold_values) / Q_vendida
    Ingreso_total = sum(sold_values)
else:
    P_promedio_vendido = 0
    Ingreso_total = 0

df_resultado = pd.DataFrame({
    'Q_vendida': [Q_vendida],
    'P_promedio_vendido': [P_promedio_vendido],
    'Ingreso_total': [Ingreso_total]
})

print(df_resultado)


In [ ]:
df_resultado

# Paciencia finita del comprador

## Fijo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def run_auction_with_bidder_patience(N, tau):
    """
    Simula una subasta donde los postores tienen una paciencia finita 'tau'.
    Si la oferta de un postor no se vende en 'tau' pasos de tiempo, se retira.

    Args:
        N (int): Número total de postores.
        tau (int): La paciencia o ventana de interés de cada postor.

    Returns:
        tuple: Contiene (ofertas vendidas, ofertas pendientes al final, ofertas expiradas).
    """
    bids = np.random.uniform(0, 1, N)
    
    pending_bids = []
    sold_bids = []
    expired_bids = [] # Nueva lista para rastrear las ofertas retiradas

    # 'i' representa el paso de tiempo actual
    for i, bid_value in enumerate(bids):
        current_time = i
        
        # --- PASO 1: EXPIRACIÓN DE OFERTAS ---
        # Antes de hacer nada, vemos si algún postor perdió la paciencia.
        bids_that_expired = [b for b in pending_bids if current_time - b['index'] >= tau]
        if bids_that_expired:
            expired_bids.extend(bids_that_expired)
            # Actualizamos la lista de pendientes, quedándonos solo con los que no expiraron
            pending_bids = [b for b in pending_bids if current_time - b['index'] < tau]

        # --- PASO 2: PROCESAR LA NUEVA OFERTA ---
        new_bid = {'bid': bid_value, 'index': i}

        # Si después de la limpieza no quedan ofertas, solo añadimos la nueva
        if not pending_bids:
            pending_bids.append(new_bid)
            continue
            
        # El campeón es el máximo de las ofertas que quedan (las que no expiraron)
        champion = max(pending_bids, key=lambda x: x['bid'])
        
        # Aplicamos la regla de m=1
        if new_bid['bid'] < champion['bid']:
            sold_bids.append(champion)
            pending_bids.remove(champion)
        
        pending_bids.append(new_bid)

    return sold_bids, pending_bids, expired_bids

# --- PARÁMETROS DE LA SIMULACIÓN ---
N = 1000
TAU = 10 # Paciencia de los postores.

# --- EJECUCIÓN ---
sold, pending_final, expired = run_auction_with_bidder_patience(N, TAU)

# --- PREPARACIÓN DE DATOS PARA GRÁFICO ---
sold_indices = [item['index'] for item in sold]
sold_values = [item['bid'] for item in sold]

# Los "No Vendidos" son tanto los que expiraron como los que quedaron pendientes al final
unsold_items = pending_final + expired
unsold_indices = [item['index'] for item in unsold_items]
unsold_values = [item['bid'] for item in unsold_items]

# --- VISUALIZACIÓN ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(16, 8))

plt.scatter(unsold_indices, unsold_values, c='black', label=f'No Vendidos ({len(unsold_items)} en total)', s=15, alpha=0.6)
plt.scatter(sold_indices, sold_values, c='red', label=f'Vendidos ({len(sold)} en total)', s=25, alpha=0.8, edgecolors='black', linewidths=0.5)

plt.title(f'Simulación con Paciencia del Postor ($\\tau$ = {TAU})', fontsize=18)
plt.xlabel('Índice del Postor (Tiempo)', fontsize=12)
plt.ylabel('Valor de la Oferta', fontsize=12)
# Mejorar la leyenda para que no esté superpuesta: usar bbox_to_anchor fuera del gráfico
plt.legend(loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=12, borderaxespad=0.)
plt.xlim(0, N)
plt.ylim(0, 1)

plt.tight_layout(rect=[0, 0, 0.85, 1])  # Deja espacio a la derecha para la leyenda
plt.show()

# Calcular Q (cantidad vendida), P promedio vendido e ingreso total
Q_vendida = len(sold)
if Q_vendida > 0:
    P_promedio_vendido = np.mean([item['bid'] for item in sold])
    Ingreso_total = np.sum([item['bid'] for item in sold])
else:
    P_promedio_vendido = 0
    Ingreso_total = 0

# Crear DataFrame con los resultados
import pandas as pd
df_resultados = pd.DataFrame({
    'Q_vendida': [Q_vendida],
    'P_promedio_vendido': [P_promedio_vendido],
    'Ingreso_total': [Ingreso_total]
})

print(df_resultados)




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parámetros
np.random.seed(42)
N   = 1000              # número total de pujas
tau = 50                # tiempo de interés (en número de pujas posteriores)
bids = np.random.uniform(0, 1, N)

# Listas de resultados
accepted_bids            = []
accepted_bid_indices     = []
non_accepted_bids        = []
non_accepted_bid_indices = []

# Pendientes: lista de tuplas (valor, índice_de_llegada)
pending = []

# Recorremos todas las pujas
for i, bid in enumerate(bids, start=1):
    # 1) Expiración de pujas pendientes que llevan >= tau pasos
    #    (es decir, su índice_de_llegada <= i - tau)
    while pending and pending[0][1] <= i - tau:
        val_expired, idx_expired = pending.pop(0)
        non_accepted_bids.append(val_expired)
        non_accepted_bid_indices.append(idx_expired)

    # 2) Si ya no quedan pendientes, simplemente metemos la nueva
    if not pending:
        pending.append((bid, i))
        continue

    # 3) Hallamos la puja máxima entre las que siguen activas
    max_val, max_idx = max(pending, key=lambda x: x[0])

    # 4) Aplicamos la regla de venta
    if bid < max_val:
        # vendemos la puja máxima
        accepted_bids.append(max_val)
        accepted_bid_indices.append(max_idx)
        pending.remove((max_val, max_idx))

        # la actual entra como pendiente (no aceptada ahora)
        non_accepted_bids.append(bid)
        non_accepted_bid_indices.append(i)
        pending.append((bid, i))
    else:
        # no se vende, acumulamos esta puja como pendiente
        non_accepted_bids.append(bid)
        non_accepted_bid_indices.append(i)
        pending.append((bid, i))

# 5) Al finalizar, todas las que queden en pending
#    habrán caducado o se congelan como no aceptadas
for val, idx in pending:
    non_accepted_bids.append(val)
    non_accepted_bid_indices.append(idx)

# --- Preparamos la gráfica ---
plt.figure(figsize=(10, 6))
plt.scatter(non_accepted_bid_indices, non_accepted_bids,
            c='black', label='No Aceptadas', s=20, alpha=0.6)
plt.scatter(accepted_bid_indices, accepted_bids,
            c='red', label='Aceptadas', s=20, alpha=0.8)

critical_price = 1/np.e
plt.axhline(y=critical_price, color='purple', linestyle='--',
            label=f'Critical Price ({critical_price:.4f})')

plt.title(f'Auction Simulation con Tiempo de Interés τ = {tau}')
plt.xlabel('Índice de Pujador')
plt.ylabel('Valor de la Puja')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

## Paciencia individual

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

N        = 1000       # nº total de pujas
tau_mean = 10         # media para el tau de cada puja
bids     = np.random.uniform(0, 1, N)

accepted_vals, accepted_idx = [], []
nonacc_vals,  nonacc_idx   = [], []

# Cada pendiente: (valor, llegada_i, tau_i)
pending = []

for j, bid in enumerate(bids, start=1):
    # 1) Generar un tau_i para la puja actual
    tau_i = np.random.exponential(scale=tau_mean)
    
    # 2) Expirar las pendientes cuyo plazo se ha cumplido
    #    (mientras hayan y la más antigua ya esté caducada)
    pending = [(v, t0, τ) for (v, t0, τ) in pending if j < t0 + τ or \
               (nonacc_vals.append(v) or nonacc_idx.append(t0) or False)]
    # nota: la list-comp descarta y a la vez registra los expirados

    # 3) Si no hay pendientes activas, la metemos directamente
    if not pending:
        pending.append((bid, j, tau_i))
        continue

    # 4) Hallar la pendiente de mayor valor activo
    v_max, t0_max, τ_max = max(pending, key=lambda x: x[0])

    # 5) Regla de venta
    if bid < v_max:
        # vendemos v_max
        accepted_vals.append(v_max)
        accepted_idx.append(t0_max)
        pending.remove((v_max, t0_max, τ_max))

        # la actual entra como pendiente no aceptada aún
        nonacc_vals.append(bid)
        nonacc_idx.append(j)
        pending.append((bid, j, tau_i))
    else:
        # no se vende, se añade como pendiente
        nonacc_vals.append(bid)
        nonacc_idx.append(j)
        pending.append((bid, j, tau_i))

# 6) Al final, lo que quede en pending caduca
for v, t0, τ in pending:
    nonacc_vals.append(v)
    nonacc_idx.append(t0)

# --- Plotting ---
plt.figure(figsize=(10,6))
sc1 = plt.scatter(nonacc_idx, nonacc_vals, c='gray', s=15, alpha=0.6, label='Pujas no aceptadas (caducadas o pendientes)')
sc2 = plt.scatter(accepted_idx, accepted_vals, c='red',  s=20, alpha=0.8, label='Pujas aceptadas (vendidas)')

cp = 1/np.e
hl = plt.axhline(cp, color='purple', linestyle='--', label=f'Precio crítico $1/e$ ≈ {cp:.3f}')
plt.title('Subasta con paciencia individual τᵢ ~ exp(τ_mean)')
plt.xlabel('Índice de puja')
plt.ylabel('Valor de la puja')

# Mejora de la leyenda: orden, descripciones y mayor claridad
from matplotlib.lines import Line2D
custom_legend = [
    Line2D([0], [0], marker='o', color='w', label='Pujas no aceptadas (caducadas o pendientes)',
           markerfacecolor='gray', markersize=8, alpha=0.6),
    Line2D([0], [0], marker='o', color='w', label='Pujas aceptadas (vendidas)',
           markerfacecolor='red', markersize=8, alpha=0.8),
    Line2D([0], [0], color='purple', lw=2, linestyle='--', label=f'Precio crítico $1/e$ ≈ {cp:.3f}')
]
plt.legend(handles=custom_legend, loc='upper center', bbox_to_anchor=(0.5, -0.12), frameon=True, fontsize=11, title="Leyenda", title_fontsize=12, ncol=1)
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()


# Calcular Q_vendida, P_promedio_vendido e Ingreso_total y poner en un DataFrame
import pandas as pd

Q_vendida = len(accepted_vals)
P_promedio_vendido = np.mean(accepted_vals) if accepted_vals else 0
Ingreso_total = np.sum(accepted_vals)

df_resultados = pd.DataFrame({
    'Q_vendida': [Q_vendida],
    'P_promedio_vendido': [P_promedio_vendido],
    'Ingreso_total': [Ingreso_total]
})

df_resultados


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

N        = 1000       # nº total de pujas
tau_mean = 10         # media para el tau de cada puja
bids     = np.random.uniform(0, 1, N)

accepted_vals, accepted_idx = [], []
nonacc_vals,  nonacc_idx   = [], []

# Cada pendiente: (valor, llegada_i, tau_i)
pending = []

for j, bid in enumerate(bids, start=1):
    # 1) Generar un tau_i para la puja actual
    tau_i = np.random.exponential(scale=tau_mean)
    
    # 2) Expirar las pendientes cuyo plazo se ha cumplido
    #    (mientras hayan y la más antigua ya esté caducada)
    pending = [(v, t0, τ) for (v, t0, τ) in pending if j < t0 + τ or \
               (nonacc_vals.append(v) or nonacc_idx.append(t0) or False)]
    # nota: la list-comp descarta y a la vez registra los expirados

    # 3) Si no hay pendientes activas, la metemos directamente
    if not pending:
        pending.append((bid, j, tau_i))
        continue

    # 4) Hallar la pendiente de mayor valor activo
    v_max, t0_max, τ_max = max(pending, key=lambda x: x[0])

    # 5) Regla de venta
    if bid < v_max:
        # vendemos v_max
        accepted_vals.append(v_max)
        accepted_idx.append(t0_max)
        pending.remove((v_max, t0_max, τ_max))

        # la actual entra como pendiente no aceptada aún
        nonacc_vals.append(bid)
        nonacc_idx.append(j)
        pending.append((bid, j, tau_i))
    else:
        # no se vende, se añade como pendiente
        nonacc_vals.append(bid)
        nonacc_idx.append(j)
        pending.append((bid, j, tau_i))

# 6) Al final, lo que quede en pending caduca
for v, t0, τ in pending:
    nonacc_vals.append(v)
    nonacc_idx.append(t0)

# --- Plotting ---
plt.figure(figsize=(10,6))
plt.scatter(nonacc_idx, nonacc_vals, c='gray', s=15, alpha=0.6, label='No aceptadas')
plt.scatter(accepted_idx, accepted_vals, c='red',  s=20, alpha=0.8, label='Aceptadas')

cp = 1/np.e
plt.axhline(cp, color='purple', linestyle='--', label=f'Critical Price={cp:.3f}')
plt.title('Subasta con τᵢ aleatorio (exp(1/τ_mean))')
plt.xlabel('Índice de puja')
plt.ylabel('Valor')
plt.legend(); plt.grid(True)
plt.show()

# Impaciencia del vendedor

Vende tras pasar 50 ofertas al candidato a pesar de no esperar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parámetros
np.random.seed(42)
N   = 1000
tau = 50                        # impaciencia: espera máxima de tau pujas
bids = np.random.uniform(0,1,N) # secuencia de pujas

# Para acumular resultados
accepted_vals  = []  # precios de venta
accepted_times = []  # momentos en que se vendió el candidato
all_times      = list(range(1, N+1))
all_vals       = bids.copy()

# Estado interno: el único candidato pendiente
pending_val  = None  # valor de la puja máxima actual
pending_time = None  # instante en que llegó dicho candidato

# Recorremos las pujas
for t, b in enumerate(bids, start=1):
    if pending_val is None:
        # primera puja: la guardamos como candidato
        pending_val, pending_time = b, t
        continue

    # 1) ¿ha expirado la paciencia?
    if t - pending_time >= tau:
        # cierro venta al precio pending_val
        accepted_vals.append(pending_val)
        accepted_times.append(pending_time)
        # el nuevo bid arranca un nuevo candidato
        pending_val, pending_time = b, t
        continue

    # 2) ¿me rebaja la nueva puja?
    if b < pending_val:
        # vendo el pending_val
        accepted_vals.append(pending_val)
        accepted_times.append(pending_time)
        # ahora el nuevo bid pasa a ser el candidato
        pending_val, pending_time = b, t
    else:
        # 3) ni vence la paciencia ni me rebaja: 
        #    reemplazo el candidato por el bid (es el nuevo máximo)
        pending_val, pending_time = b, t

# Al final, el candidato que quede nunca se vende
# (podrías decidir que expire también, pero aquí lo tratamos como no‐vendido)

# Preparo listas de no‐aceptadas
nonacc_times = [t for t in all_times if t not in accepted_times]
nonacc_vals  = [all_vals[t-1] for t in nonacc_times]

# ====== Plot ======
plt.figure(figsize=(10,6))
plt.scatter(nonacc_times, nonacc_vals,
            c='black', s=20, alpha=0.6, label='No vendidas')
plt.scatter(accepted_times, accepted_vals,
            c='red',   s=20, alpha=0.8, label='Vendidas (aceptadas)')

plt.title(f'Subasta M=1 + impaciencia τ={tau}')
plt.xlabel('Índice de puja')
plt.ylabel('Valor de la puja')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()
plt.show()

# Impaciencia con precio de reserva

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set a seed for reproducibility
np.random.seed(42)

# Parameters
N = 1000
decay_rate = 0.05  # Factor de decaimiento de la impaciencia

# Generate a sequence of 1000 bids from a uniform distribution
bids = np.random.uniform(0, 1, N)

# Initialize variables
accepted_bids = []
non_accepted_bids = []
accepted_bid_indices = []
non_accepted_bid_indices = []
pending_bids = []

# Variables for impatience tracking
last_sale_index = 0
sales_count = 0
impatience_thresholds = []

def calculate_impatience_threshold(current_index, last_sale_index, avg_bids, sales_count):
    """
    Calcula el umbral de impaciencia que va decayendo:
    - Arranca en 1/e
    - Después se basa en promedio de bids con decaimiento exponencial
    """
    if sales_count == 0:
        # Primera venta: usar 1/e
        threshold = 1/np.e
    else:
        # Tiempo transcurrido desde la última venta
        time_since_last_sale = current_index - last_sale_index
        
        # Factor de decaimiento exponencial (más tiempo = más impaciencia = umbral más bajo)
        decay_factor = np.exp(-decay_rate * time_since_last_sale)
        
        # Umbral = promedio de bids × decaimiento
        threshold = avg_bids * decay_factor
        
        # Asegurar que no sea muy bajo
        threshold = max(threshold, 0.05)
    
    return threshold

# Implement the auction mechanism with seller impatience
for i, new_bid in enumerate(bids):
    bid_index = i + 1
    
    # Si no hay bids pendientes, agregar el nuevo bid
    if not pending_bids:
        pending_bids.append(new_bid)
        non_accepted_bids.append(new_bid)
        non_accepted_bid_indices.append(bid_index)
        continue
    
    # Encontrar el bid máximo entre los pendientes
    max_pending_bid = max(pending_bids)
    
    # Calcular umbral de impaciencia
    avg_bids_so_far = np.mean(bids[:i])
    impatience_threshold = calculate_impatience_threshold(
        bid_index, last_sale_index, avg_bids_so_far, sales_count
    )
    impatience_thresholds.append(impatience_threshold)
    
    # Reglas de venta:
    sell = False
    
    # ESTRUCTURA ORIGINAL: Si llega un bid menor al máximo pendiente
    if new_bid < max_pending_bid:
        # Regla 1: Si ya pasó al menos 1 transacción, vender
        if sales_count >= 1:
            sell = True
        # Regla 2: Si el máximo pendiente supera el umbral de impaciencia
        elif max_pending_bid >= impatience_threshold:
            sell = True
    
    if sell:
        # Venta ocurre - SIEMPRE al mayor bid pendiente
        accepted_bid_index = np.where(bids == max_pending_bid)[0][0] + 1
        accepted_bids.append(max_pending_bid)
        accepted_bid_indices.append(accepted_bid_index)
        
        # Actualizar contadores
        last_sale_index = bid_index
        sales_count += 1
        
        # Remover el bid aceptado de pendientes
        pending_bids.remove(max_pending_bid)
        
        # El bid actual se convierte en pendiente
        pending_bids.append(new_bid)
        non_accepted_bids.append(new_bid)
        non_accepted_bid_indices.append(bid_index)
    else:
        # No hay venta, agregar a pendientes
        pending_bids.append(new_bid)
        non_accepted_bids.append(new_bid)
        non_accepted_bid_indices.append(bid_index)

# Los bids restantes quedan como no aceptados
for bid in pending_bids:
    if bid not in non_accepted_bids:
        non_accepted_bids.append(bid)
        non_accepted_bid_indices.append(np.where(bids == bid)[0][0] + 1)

# Preparar datos para el gráfico
accepted_indices_plot = [i for i, b in enumerate(bids) if b in accepted_bids]
non_accepted_indices_plot = [i for i, b in enumerate(bids) if b not in accepted_bids]

accepted_values_plot = [bids[i] for i in accepted_indices_plot]
non_accepted_values_plot = [bids[i] for i in non_accepted_indices_plot]

accepted_indices_plot = [i + 1 for i in accepted_indices_plot]
non_accepted_indices_plot = [i + 1 for i in non_accepted_indices_plot]

# Create the enhanced plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Primer gráfico: Simulación de subasta
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

ax1.scatter(non_accepted_indices_plot, non_accepted_values_plot, 
           c='black', label='Not Accepted', s=20, alpha=0.3)
ax1.scatter(accepted_indices_plot, accepted_values_plot, 
           c='red', label='Accepted', s=30, alpha=0.7)

# Add critical price line
critical_price = 1/np.e
ax1.axhline(y=critical_price, color='#800080', linestyle=':', linewidth=3.5, 
            dashes=(8, 3), alpha=0.9, label='1/e threshold')

ax1.text(1060, critical_price, f'1/e ≈ {round(critical_price, 4)}', 
         color='#800080', fontsize=11, verticalalignment='center')

ax1.set_title('Simulación de subasta con impaciencia del vendedor\n' +
             f'Se vende al mayor bid cuando llega uno menor. Total de ventas: {len(accepted_bids)}',
             fontsize=14, fontweight='bold', pad=20)
ax1.set_xlabel('Bidder')
ax1.set_ylabel('Bid [$]')
ax1.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=10)

# Segundo gráfico: Evolución del umbral de impaciencia
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

if impatience_thresholds:
    x_threshold = list(range(2, len(impatience_thresholds) + 2))
    ax2.plot(x_threshold, impatience_thresholds, color='orange', linewidth=2, alpha=0.8)

ax2.set_title('Umbral de impaciencia del vendedor\n' +
             f'Decae exponencialmente desde cada venta (decay_rate={decay_rate})',
             fontsize=14, fontweight='bold', pad=20)
ax2.set_xlabel('Bidder')
ax2.set_ylabel('Impatience Threshold [$]')

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"\nResumen de la simulación:")
print(f"Total de bids: {N}")
print(f"Bids aceptados: {len(accepted_bids)}")
print(f"Tasa de aceptación: {len(accepted_bids)/N:.1%}")
print(f"Precio promedio de venta: ${np.mean(accepted_bids):.4f}")
print(f"Umbral final de impaciencia: ${impatience_thresholds[-1] if impatience_thresholds else 1/np.e:.4f}")
print(f"Rango del umbral: ${min(impatience_thresholds) if impatience_thresholds else 1/np.e:.4f} - ${max(impatience_thresholds) if impatience_thresholds else 1/np.e:.4f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Set a seed for reproducibility
np.random.seed(42)

# Parameters
N = 1000
decay_rate = 0.05  # Factor de decaimiento de la impaciencia

# Generate a sequence of 1000 bids from a uniform distribution
bids = np.random.uniform(0, 1, N)

# Initialize variables
accepted_bids = []
accepted_bid_indices = []
pending_bids = []

# Variables for impatience tracking
last_sale_index = 0
sales_count = 0
impatience_thresholds = []
max_pending_bids_history = []
sale_events_x = []
sale_events_y = []

def calculate_impatience_threshold(current_index, last_sale_index, avg_bids, sales_count):
    """
    Calcula el umbral de impaciencia que va decayendo:
    - Arranca en 1/e
    - Después se basa en promedio de bids con decaimiento exponencial
    """
    if sales_count == 0:
        threshold = 1/np.e
    else:
        time_since_last_sale = current_index - last_sale_index
        decay_factor = np.exp(-decay_rate * time_since_last_sale)
        threshold = avg_bids * decay_factor
        threshold = max(threshold, 0.05)
    return threshold

# Implement the auction mechanism with seller impatience
for i, new_bid in enumerate(bids):
    bid_index = i + 1

    # Si no hay bids pendientes, agregar el nuevo bid
    if not pending_bids:
        pending_bids.append(new_bid)
        max_pending_bids_history.append(new_bid)
        continue

    # Encontrar el bid máximo entre los pendientes
    max_pending_bid = max(pending_bids)
    max_pending_bids_history.append(max_pending_bid)

    # Calcular umbral de impaciencia
    avg_bids_so_far = np.mean(bids[:i]) if i > 0 else new_bid
    impatience_threshold = calculate_impatience_threshold(
        bid_index, last_sale_index, avg_bids_so_far, sales_count
    )
    impatience_thresholds.append(impatience_threshold)

    # Reglas de venta (OR lógico):
    sell = False
    if new_bid < max_pending_bid:
        sell = True
    elif max_pending_bid >= impatience_threshold:
        sell = True

    if sell:
        # Venta ocurre - SIEMPRE al mayor bid pendiente
        accepted_bid_index = np.where(bids == max_pending_bid)[0][0] + 1
        accepted_bids.append(max_pending_bid)
        accepted_bid_indices.append(accepted_bid_index)
        sale_events_x.append(bid_index)
        sale_events_y.append(max_pending_bid)
        last_sale_index = bid_index
        sales_count += 1
        pending_bids.remove(max_pending_bid)
        pending_bids.append(new_bid)
    else:
        pending_bids.append(new_bid)

# --- DataFrame con resumen ---
Q = len(accepted_bids)
P_promedio = np.mean(accepted_bids) if accepted_bids else 0
Ingreso_total = np.sum(accepted_bids) if accepted_bids else 0
umbral_min = min(impatience_thresholds) if impatience_thresholds else 1/np.e
umbral_max = max(impatience_thresholds) if impatience_thresholds else 1/np.e
umbral_promedio = np.mean(impatience_thresholds) if impatience_thresholds else 1/np.e

df_resumen = pd.DataFrame({
    "Q_vendida": [Q],
    "P_promedio_vendido": [P_promedio],
    "Ingreso_total": [Ingreso_total],
    "Umbral_min": [umbral_min],
    "Umbral_max": [umbral_max],
    "Umbral_promedio": [umbral_promedio]
})

print("\nResumen de la simulación:\n")
print(df_resumen)

# --- Identificar los no aceptados ---
# accepted_bid_indices son 1-based, bids es 0-based
accepted_bid_indices_set = set(accepted_bid_indices)
all_indices = np.arange(1, N+1)
non_accepted_indices = [i for i in all_indices if i not in accepted_bid_indices_set]
non_accepted_values = [bids[i-1] for i in non_accepted_indices]

# --- Gráfico de líneas suavizado ---
def moving_average(x, w):
    return np.convolve(x, np.ones(w)/w, mode='valid')

window_size = 25  # Ajustar para más/menos suavizado
if len(impatience_thresholds) >= window_size:
    x_threshold = np.arange(2, len(impatience_thresholds) + 2)
    smooth_umbral = moving_average(impatience_thresholds, window_size)
    x_smooth_umbral = x_threshold[window_size-1:]
else:
    smooth_umbral = impatience_thresholds
    x_smooth_umbral = np.arange(2, len(impatience_thresholds) + 2)

# --- Gráficos ---
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Primer gráfico: scatter accepted (rojo) y no aceptados (negro)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['left'].set_visible(True)
ax1.spines['bottom'].set_visible(True)
ax1.grid(False)

ax1.scatter(non_accepted_indices, non_accepted_values, 
            c='black', label='Not Accepted', s=20, alpha=0.3)
ax1.scatter(accepted_bid_indices, accepted_bids, 
            c='red', label='Accepted', s=30, alpha=0.7)

critical_price = 1/np.e
ax1.axhline(y=critical_price, color='#800080', linestyle=':', linewidth=3.5, 
            dashes=(8, 3), alpha=0.9, label='1/e threshold')
ax1.text(1060, critical_price, f'1/e ≈ {round(critical_price, 4)}', 
         color='#800080', fontsize=11, verticalalignment='center')

ax1.set_title('Simulación de subasta con impaciencia del vendedor\n' +
             f'Se vende al mayor bid cuando llega uno menor. Total de ventas: {Q}',
             fontsize=14, fontweight='bold', pad=20)
ax1.set_xlabel('Bidder')
ax1.set_ylabel('Bid [$]')
ax1.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=10)

# Segundo gráfico: solo el umbral suavizado
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(True)
ax2.spines['bottom'].set_visible(True)
ax2.grid(False)

ax2.plot(x_smooth_umbral, smooth_umbral, color='orange', linewidth=2, alpha=0.9, label='Umbral de impaciencia (suavizado)')

ax2.set_title('Evolución suavizada del umbral de impaciencia\n' +
             f'Decae exponencialmente desde cada venta (decay_rate={decay_rate})',
             fontsize=14, fontweight='bold', pad=20)
ax2.set_xlabel('Bidder')
ax2.set_ylabel('Umbral [$]')
ax2.legend(loc='upper right', fontsize=10, frameon=True)

plt.tight_layout()
plt.show()

In [ ]:
df_resumen